In [1]:
import networkx as nx

In [2]:
G = nx.read_gpickle('Minto Flats Network with Metadata.pkl')
G.nodes(data=True)[0]

('A/shorebird/Delaware Bay/331/2008',
 {'band_num': None,
  'cloacal_id': None,
  'host': 'IRD:Shorebird/Avian',
  'isolation_date': Timestamp('2008-05-17 00:00:00'),
  'oral_id': None,
  'state': 'Delaware',
  'subtype': '(N6)',
  'web_tag': None})

In [3]:
G.edges(data=True)[0]

('A/American black duck/Wisconsin/11OS3191/2011',
 'A/mallard/Wisconsin/11OS3170/2011',
 {'5season_transmission': 'Fall-Fall',
  'family_transmission': 'Intrafamily',
  'foraging_transmission': 'Intraforaging',
  'order_transmission': 'Intra-Order',
  'species_transmission': 'Interspecies',
  'time_delta': datetime.timedelta(0),
  'tr_timeclass': 'Direct Transmission',
  'trtype': 'Inter-Species',
  'weight': 7.962964,
  'weighting_timedelta': 0.5})

In [32]:
# Modifications to the fields begin here.
# The "weight" field should be "pwi" field. 
for sc, sk, d in G.edges(data=True):
    G.edge[sc][sk]['pwi'] = d['weight']

for n, d in G.nodes(data=True):
    G.node[n]['collection_date'] = d['isolation_date']

In [33]:
nx.write_gpickle(G, 'Minto_Flats Full Complement Graph.pkl')

In [34]:
# Create a version of the graph that is "initialized" - that is, without edges.
cleanedG = G.copy()
cleanedG.remove_edges_from(G.edges())
nx.write_gpickle(cleanedG, 'Minto_Flats Initialized Graph.pkl')

In [31]:
cleanedG.nodes(data=True)[0]

('A/shorebird/Delaware Bay/331/2008',
 {'band_num': None,
  'cloacal_id': None,
  'host': 'IRD:Shorebird/Avian',
  'isolation_date': Timestamp('2008-05-17 00:00:00'),
  'oral_id': None,
  'state': 'Delaware',
  'subtype': '(N6)',
  'web_tag': None})

In [36]:
# Need to convert segment affmats into a single .h5 store. Also need to convert accession numbers into strain names.
import pandas as pd
data = pd.read_csv('Raw Data/20140211 Minto and North America Combined Dates Cleaned.csv', index_col=0, parse_dates=['Collection Date']).reindex()

# Create a dictionary of accession to strain name.
for i in range(1,9):
    df = pd.read_csv('2.1 - Segment Transmission/Segment {0} Transmissions.csv'.format(i), index_col='Strain Name')
#     if i == 1:
#         accession_strain = dict(zip(data['Sequence Accession'], data['Strain Name']))
#     else:
#         accession_strain = dict(zip(data['Sequence Accession{0}'.format(i)], data['Strain Name']))
#     /df['0'] = df['0'].apply(lambda x: accession_strain[x])
#     df.set_index('0', inplace=True)
#     df.columns = df.index
    
    df.to_hdf('Minto_Flats Segment Affmats.h5', mode='a', key='segment{0}'.format(i))
    